In [1]:
#pandasインポートとデータセット読み込み
import pandas as pd
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')


In [3]:
#ダミー変数化処理用関数

def dummylize(df,n_item):
    dum = pd.get_dummies(df[n_item], drop_first = True)
    df = pd.concat((df, dum),axis = 1)
    df = df.drop(n_item,axis = 1)
    return df


#nullを0、それ以外を1とする関数
def booleanize(df,n_item):
    df_tmp = df[n_item].notnull() * 1
    df_tmp.name = n_item

    df = df.drop(n_item,axis = 1)
    df = pd.concat((df, df_tmp), axis = 1)
    return df


In [4]:
#性別、乗車場所をダミー変数化、CabinをBoolean化

df_train = dummylize(df_train,'Sex')
df_train = dummylize(df_train,'Embarked')
df_train = booleanize(df_train,'Cabin')


df_test = dummylize(df_test,'Sex')
df_test = dummylize(df_test,'Embarked')
df_test = booleanize(df_test,'Cabin')


In [5]:
#Title列、Title数値化列、FamilyName列、FamilyName頻度列を追加する処理関数


#Listを含まれるタイトルのみの文字列に変換するサブ関数

def titleCheck(L):
        if 'Miss' in L: return 'Miss'
        elif 'Mrs' in L: return 'Mrs'
        elif 'Master' in L: return 'Master'
        elif 'Mr' in L: return 'Mr'
        else: return ""


#処理関数メイン

def nameAnalysis(df):

    l_names = [x.replace(",","").replace(".","").split(" ") for x in df.Name.values.tolist()]
#Mr. Mrs. などを仕分け
    l_title = [titleCheck(x) for x in l_names]
    df_title = pd.Series(l_title, name='Titles')

#Mr. Mrs. などを数値化
    from sklearn.preprocessing import LabelEncoder

    le = LabelEncoder()
    num_title = le.fit_transform(l_title)
    df_n_title = pd.Series(num_title, name='TitlesNum')

#ファミリーネーム抜出し

    l_family = [x[0] for x in l_names]
    df_family = pd.Series(l_family, name = "Family")

#ファミリーネームの出現回数列を追加
    df_n_family = df_family.apply(lambda x: (df_family == x).sum())
    df_n_family.name = 'FamilyNum'

#作成した各分析列を追加しリターン
    df = pd.concat((df, df_title), axis = 1)
    df = pd.concat((df, df_n_title), axis = 1)
    df = pd.concat((df, df_family), axis = 1)
    df = pd.concat((df, df_n_family), axis = 1)
        
    return df

In [6]:
#名前処理関数をApply

df_train = nameAnalysis(df_train)

df_test = nameAnalysis(df_test)


In [7]:
#年齢を世代区分け


def ageGroup(x):
        if 0 <= x and x < 10:
            return 0
        elif 10 <= x and x < 20:
            return 1
        elif 20 <= x and x < 40:
            return 2
        elif 40 <= x and x < 60:
            return 3
        elif 60 <= x:
            return 4


df_train.Age = df_train.Age.apply(lambda x: ageGroup(x))
df_test.Age = df_test.Age.apply(lambda x: ageGroup(x))

In [8]:
df_test.head()

,PassengerId,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,male,Q,S,Cabin,Titles,TitlesNum,Family,FamilyNum
0,892,3,"Kelly, Mr. James",2.0,0,0,330911,7.8292,1,1,0,0,Mr,3,Kelly,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",3.0,1,0,363272,7.0000,0,0,1,0,Mrs,4,Wilkes,1
2,894,2,"Myles, Mr. Thomas Francis",4.0,0,0,240276,9.6875,1,1,0,0,Mr,3,Myles,1
3,895,3,"Wirz, Mr. Albert",2.0,0,0,315154,8.6625,1,0,1,0,Mr,3,Wirz,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",2.0,1,1,3101298,12.2875,0,0,1,0,Mrs,4,Hirvonen,1


In [9]:
#年齢推定モデルに渡すためテスト用加工済みCSVを保存

df_train.to_csv('forAgePred.csv')



In [10]:
#別NoteBookの処理で作成・保存した年齢推定モデルの読み込み

import pickle
filename = 'age_pred.sav'
model_age_pred = pickle.load(open(filename, 'rb'))

In [11]:
#Age欠損部分を年齢推定モデルを使い補完する処理まとめ関数

def fillMissingAge(df):

    #モデルを使って予測し結果をリストに保存→Seriesに変換
    l_pred = ['TitlesNum']
    res_pred = model_age_pred.predict(df[l_pred])
    df_res_pred =pd.Series(res_pred, name='prediction')
    df_res_pred = df_res_pred.apply(lambda x: float(x/100))

    df_real_age = df['Age']
    df_real_age = df_real_age.fillna(0)


    #もともとのAge列の欠損値のみPredictの結果で置き換え　(もっといいやり方があるはず)

    df_real_age = df['Age']
    df_real_age = df_real_age.fillna(0)

    l_mix = []

    for i,v in enumerate(df_real_age):
        if v != 0:
            l_mix.append(v)
        else:
            l_mix.append(df_res_pred[i])

    df_mix = pd.Series(l_mix, name='Age')

    #Age列を差し替え
    df = df.drop('Age',axis = 1)
    df = pd.concat((df,df_mix),axis = 1)

    return df

In [12]:

#Age列欠損補完をApply

df_train = fillMissingAge(df_train)

df_test = fillMissingAge(df_test)


In [13]:
#XGBoostモデル

def get_xgb_model3(X, Y):
    import xgboost as xgb

    import scipy.stats as st
    from sklearn.model_selection import RandomizedSearchCV

    one_to_left = st.beta(10, 1)
    from_zero_positive = st.expon(0, 50)

    params = {
        "n_estimators": st.randint(3, 40),
        "max_depth": st.randint(3, 40),
        "learning_rate": st.uniform(0.05, 0.4),
        "colsample_bytree": one_to_left,
        "subsample": one_to_left,
        "gamma": st.uniform(0, 10),
        'reg_alpha': from_zero_positive,
        "min_child_weight": from_zero_positive,
    }

    xgbreg = xgb.XGBRegressor(nthreads=-1)


    gs = RandomizedSearchCV(xgbreg, params, n_jobs=3)
    res = gs.fit(X, Y)
    print(gs.best_params_)

    return res


In [58]:
df_train.describe()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare,male,Q,S,Cabin,TitlesNum,FamilyNum,Age
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.523008,0.381594,32.204208,0.647587,0.086420,0.722783,0.228956,2.755331,1.906846,2.014590
std,257.353842,0.486592,0.836071,1.102743,0.806057,49.693429,0.477990,0.281141,0.447876,0.420397,0.858966,1.497289,0.742612
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,2.000000
50%,446.000000,0.000000,3.000000,0.000000,0.000000,14.454200,1.000000,0.000000,1.000000,0.000000,3.000000,1.000000,2.000000
75%,668.500000,1.000000,3.000000,1.000000,0.000000,31.000000,1.000000,0.000000,1.000000,0.000000,3.000000,2.000000,2.000000
max,891.000000,1.000000,3.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,4.000000


In [69]:
#学習・テスト用データセット整備

from sklearn.model_selection import train_test_split

df_train = df_train.fillna(0)
l_features = ['Pclass','male','Q','S','TitlesNum','FamilyNum','Cabin','Age','SibSp','Parch','Fare']
df_x_pack = df_train[l_features]
df_y_pack = df_train.Survived

X_train, X_test, y_train, y_test = train_test_split(df_x_pack, df_y_pack, test_size=0.25)



In [70]:
#XGBモデル学習

model_xgb = get_xgb_model3(X_train, y_train)


{'colsample_bytree': 0.98885496749526691, 'gamma': 1.6450776900591779, 'learning_rate': 0.33858679959384752, 'max_depth': 28, 'min_child_weight': 0.42679086197347438, 'n_estimators': 29, 'reg_alpha': 0.61710364540356455, 'subsample': 0.9697640611035161}


In [85]:
#Grid Search Cross Validation適用用関数

def applyGSCV(model, param, X, Y):
    from sklearn.model_selection import GridSearchCV

    res = GridSearchCV(model, param, cv=3)
    res.fit(X, Y)

    return res

In [82]:
#Random Forestモデル学習

from sklearn.ensemble import RandomForestClassifier

obj_param = {
'n_estimators': [5,10,20,30,50,100,300],
'max_depth': [3,5,10,15,20,25,30,40,50,100],
'random_state': [0]
}

model_randForest = applyGSCV(RandomForestClassifier(),obj_param,X_train, y_train)

print('done')

In [83]:
from sklearn.neural_network import MLPClassifier
model_MLPC =  MLPClassifier(solver='lbfgs', random_state=0).fit(X_train, y_train)

In [84]:
#model 比較

from sklearn.metrics import mean_squared_error

models =[]
models.append(model_xgb)
models.append(model_randForest)
models.append(model_MLPC)

for model in models:    
    res_tmp = model.predict(X_test)
    res_mean = mean_squared_error(y_test, res_tmp)
    print('mean squared error:{0} '.format(res_mean))


mean squared error:0.12818037820014203 
mean squared error:0.17040358744394618 
mean squared error:0.19730941704035873 


In [68]:

model_final = model_xgb


{'colsample_bytree': 0.74436301228265755, 'gamma': 6.1772715269608813, 'learning_rate': 0.15496312978223037, 'max_depth': 10, 'min_child_weight': 101.83755392360787, 'n_estimators': 12, 'reg_alpha': 0.87831901557616898, 'subsample': 0.64694260899176426}


In [61]:
#モデル保存

import pickle
filename = 'xgb_model.sav'
pickle.dump(model_final, open(filename, 'wb'))

loaded_m_xgb = pickle.load(open(filename, 'rb'))

In [62]:

from sklearn.metrics import mean_squared_error

res_xgb = loaded_m_xgb.predict(X_test)
result = mean_squared_error(y_test, res_xgb)


In [63]:
result

0.16175879616873748

In [64]:
dig_res_xgb = [0 if x <=0.5 else 1 for x in res_xgb]
dig_res_xgb

print(sum(dig_res_xgb == y_test) / len(dig_res_xgb))


0.793721973094


In [65]:
df_X_forSubmit = df_test[l_features]
Y_forSubmit = loaded_m_xgb.predict(df_X_forSubmit)
Y_forSubmit_int = [0 if x <=0.5 else 1 for x in Y_forSubmit]

df_Y = pd.Series(Y_forSubmit_int, name='Survived')
df_submit = df_test.PassengerId.copy()

df_submit = pd.concat((df_submit,df_Y),axis=1)


In [66]:
df_submit.head()
df_submit.to_csv('submission.csv', index = False)